In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
from itertools import product
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec

# Functions

In [2]:
# Define the size categories
def classify_radius(size):
    if size <= 15:
        return 0.1  # Small size
    elif 16 <= size <= 140:
        return 0.3 # Medium size
    else:
        return 0.5  # Large size

In [3]:
# Add jitter to coordinates to reduce overlap
def jitter_coordinates(lat, lon, max_jitter=3.5):
    jittered_lat = lat + np.random.uniform(-max_jitter, max_jitter)
    jittered_lon = lon + np.random.uniform(-max_jitter, max_jitter)
    return jittered_lat, jittered_lon

# Specifications

In [4]:
optimal_cluster = 'default'
name = ''
save = False
save_data = False

if optimal_cluster == 'default':
    optimal_cluster = 'st_cluster_3_5_7'
    name = ''
    folder = ''
else:
    folder = 'SI/'

# Load Data

In [5]:
claims_clusters = pd.read_csv('clustered_claims_final.csv')

/tmp/ipykernel_2811/935040874.py:1: DtypeWarning: Columns (8,26,33,36,39,43,52,53,62) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_clusters = pd.read_csv('clustered_claims_final.csv')


In [6]:
# 1) Coerce dtypes
num_cols = ["latitude", "longitude", "adjustedClaim"]
claims_clusters[num_cols] = claims_clusters[num_cols].apply(pd.to_numeric, errors="coerce")
claims_clusters["dateOfLoss"] = pd.to_datetime(claims_clusters["dateOfLoss"], errors="coerce")
claims_clusters['countyCode'] = claims_clusters['countyCode'].astype(int).astype(str)
claims_clusters['countyCode'] = claims_clusters['countyCode'].apply(lambda x: str(x).zfill(5))

In [7]:
sum_hist = 831697554.47 # From NFIP Historic Policies
sum_2024 = 3879323216 # From NFIP RR2 Policies

# Geospatial Data

In [8]:
# Load the county shapefile
county_shapefile_path = '../Local_Data/Geospatial/tl_2019_us_county.shp'
gdf_counties = gpd.read_file(county_shapefile_path)

# Load the shapefile for US states
state_shapefile_path = '../Local_Data/Geospatial/cb_2018_us_state_20m.shp'
gdf_states = gpd.read_file(state_shapefile_path)

# Using Storm Clustering

In [9]:
# Group by optimal_cluster and calculate medians and sizes
cluster_centers = claims_clusters.groupby(optimal_cluster).agg(
    median_latitude=('latitude', 'median'),
    median_longitude=('longitude', 'median'),
    cluster_size=('latitude', 'size'),  # Count rows in each cluster
    claim_sum=('adjustedClaim', 'sum'), # Median of the claim_sum for color mapping
    median_date=("dateOfLoss", "median") 
).reset_index()

# Calculate the 90th and 50th percentiles of the cluster_size
percentile_90 = np.percentile(cluster_centers['cluster_size'], 90)
percentile_50 = np.percentile(cluster_centers['cluster_size'], 50)

# Print the results
print(f"90th Percentile of cluster sizes: {percentile_90}")
print(f"50th Percentile (Median) of cluster sizes: {percentile_50}")

90th Percentile of cluster sizes: 141.80000000000018
50th Percentile (Median) of cluster sizes: 15.0


## Calculations of Interest

In [10]:
# Sort by claim_sum value to plot higher values last
cluster_centers = cluster_centers.sort_values(by='claim_sum', ascending=True)

# Classify cluster sizes into the three categories
cluster_centers['radius'] = cluster_centers['cluster_size'].apply(classify_radius)

sum_unclustered = claims_clusters[claims_clusters[optimal_cluster] == -1]['adjustedClaim'].sum()
annual_unclustered = float(sum_unclustered)/(claims_clusters['yearOfLoss'].max()-claims_clusters['yearOfLoss'].min())

average_claim_sum = cluster_centers['claim_sum'].mean()

# Total sum of 'claim_sum'
total_claim_sum = cluster_centers['claim_sum'].sum()
top_10_sum = cluster_centers.nlargest(10, 'claim_sum')['claim_sum'].sum()
top_8_sum = cluster_centers.nlargest(8, 'claim_sum')['claim_sum'].sum()

# Sum of all others except the top 20
sum_except_top_10 = total_claim_sum - top_10_sum
sum_except_top_8 = total_claim_sum - top_8_sum

# Extract year from the 'median_date' column
cluster_centers['year'] = pd.to_datetime(cluster_centers['median_date']).dt.year

# Group by year and count clusters
clusters_per_year = cluster_centers.groupby('year')[optimal_cluster].nunique()

# Calculate the average number of clusters per year
average_clusters_per_year = clusters_per_year.mean()

# Sort the DataFrame by 'claim_sum' in descending order
cluster_centers = cluster_centers.sort_values(by='claim_sum', ascending=False)

# Select the top 8 rows (99.9%)
top_8_clusters = cluster_centers.head(8)

# Assign names to the rows
top_8_clusters['Event'] = ["Hurricane Katrina", "Hurricane Sandy", "Hurricane Harvey", "Hurricane Ian", "Hurricane Ike", "2016 Louisiana Flooding", "Hurricanes Ivan/Jeanne", "Hurricane Helene"]

/tmp/ipykernel_2811/2707086539.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_8_clusters['Event'] = ["Hurricane Katrina", "Hurricane Sandy", "Hurricane Harvey", "Hurricane Ian", "Hurricane Ike", "2016 Louisiana Flooding", "Hurricanes Ivan/Jeanne", "Hurricane Helene"]


In [11]:
# Data for the first chart
df1 = pd.DataFrame({
    "Type": ["Unclustered", "Mean\nCluster"],
    "Value": [annual_unclustered/1e6, average_claim_sum/1e6]
})

# Data for the second chart
df2 = pd.DataFrame({
    "Type": ["99.9% Hyperclusters", "All Other Claims"],
    "Value": [top_8_sum/1e9, sum_except_top_8/1e9]
})

# --- Top 8 ---
top_8_clusters = top_8_clusters.copy().reset_index(drop=True)
if len(top_8_clusters) != 8:
    raise ValueError(f"Expected 8 rows in top_8_clusters, found {len(top_8_clusters)}")

top_8_clusters["Event"] = [
    "Hurricane\nKatrina", "Hurricane\nSandy", "Hurricane\nHarvey", "Hurricane\nIan",
    "Hurricane\nIke", "Louisiana\nFlood 2016", "Hurricanes\nIvan+Jeanne", "Hurricane\nHelene"
]
top_8_clusters["claim_sum"] = pd.to_numeric(top_8_clusters["claim_sum"], errors="coerce") / 1e9

In [12]:
# Sort the DataFrame by 'claim_sum' in descending order
cluster_centers = cluster_centers.sort_values(by='claim_sum', ascending=False)

# Select the top 12 rows
billion_dollar = cluster_centers[cluster_centers[optimal_cluster] != -1].head(12)

# Sort the DataFrame by 'claim_sum' in ascending order
billion_dollar = billion_dollar.sort_values(by='claim_sum', ascending=True)

# Extract year from 'median_date' and add it as a new column
billion_dollar['year'] = pd.to_datetime(billion_dollar['median_date']).dt.year

# Define radius using log scale for 'claim_sum'
min_claim_sum = billion_dollar['claim_sum'].min()
log_base = 10  # Adjust the base for scaling if needed
max_radius = 3  # Maximum radius for visualization
billion_dollar['radius'] = (
    billion_dollar['cluster_size'] / billion_dollar['cluster_size'].max() * max_radius
)

# Apply jittering to each cluster's coordinates
billion_dollar[['jittered_latitude', 'jittered_longitude']] = billion_dollar.apply(
    lambda row: jitter_coordinates(row['median_latitude'], row['median_longitude']), axis=1, result_type="expand"
)

In [ ]:
# Define the overall figure with a grid layout
fig = plt.figure(figsize=(12, 8), constrained_layout=True)
gs = GridSpec(2, 2, width_ratios=[2, 1])  # 2 rows, 2 columns with narrow barplot column

# Limit to contiguous US (approximate bounding box)
extent = [-125, -66.5, 24, 49]  # [min_lon, max_lon, min_lat, max_lat]

# First column: Maps
ax_map1 = fig.add_subplot(gs[0, 0])
ax_map2 = fig.add_subplot(gs[1, 0])

# Second column: Nested GridSpec for bar plots
gs_bar = GridSpecFromSubplotSpec(2, 1, subplot_spec=gs[:, 1], height_ratios=[1, 1], hspace=0.3)  # Adjusted hspace
bar_axes = [fig.add_subplot(gs_bar[i, 0]) for i in range(2)]  # Create individual bar plot axes

# Sort by claim_sum value to plot higher values last
cluster_centers = cluster_centers.sort_values(by='claim_sum', ascending=True)

# Determine the min and max of the claim_sum
cluster_centers['claim_sum'] = cluster_centers['claim_sum'].replace([np.inf, -np.inf], np.nan)
cluster_centers['claim_sum'].fillna(1e4, inplace=True)  # Replace NaN with a small value (e.g., 1e-1)
cluster_centers['claim_sum'] = cluster_centers['claim_sum'].apply(lambda x: max(x, 1e4))  # Replace with a small positive number
vmin = cluster_centers['claim_sum'].min()
vmax = cluster_centers['claim_sum'].max()

# Update color map to yellow-to-red
cmap_yellow_red = plt.cm.YlOrRd

# Log normalization for color mapping
norm = mcolors.LogNorm(vmin=vmin, vmax=vmax)

# Map the claim_sum to colors
cluster_centers['color'] = cluster_centers['claim_sum'].apply(lambda x: cmap_yellow_red(norm(x)) if not np.isnan(x) else 'gray')

# Plot 1: Map of all clusters
gdf_counties.plot(ax=ax_map1, color='lightgray', edgecolor='gray', alpha=0.5)
gdf_states.boundary.plot(ax=ax_map1, color='black', linewidth=0.5)

patches1 = []
for _, row in cluster_centers.iterrows():
    center = (row['median_longitude'], row['median_latitude'])
    radius = row['radius']
    color = row['color']
    circle = Circle(center, radius, edgecolor=color, facecolor=color, alpha=0.6)
    patches1.append(circle)

p1 = PatchCollection(patches1, match_original=True, transform=ax_map1.transData)
ax_map1.add_collection(p1)

ax_map1.set_xlim(extent[0], extent[1])
ax_map1.set_ylim(extent[2], extent[3])
ax_map1.axis('off')
ax_map1.set_title("Clustered Claims", fontsize=16)

# Add subplot label "a)" in the upper left
ax_map1.text(-0.1, 1, "a)", transform=ax_map1.transAxes, fontsize=14, va='top', ha='left')

# Add legend for point sizes with varying sizes
legend_elements = [
    Line2D([0], [0], marker='o', color='gray', markersize=4, label='Size ≤ 15', linestyle='None', alpha=0.5),
    Line2D([0], [0], marker='o', color='gray', markersize=8, label='15 < Size < 140', linestyle='None', alpha=0.5),
    Line2D([0], [0], marker='o', color='gray', markersize=12, label='Size ≥ 140', linestyle='None', alpha=0.5),
]

legend = ax_map1.legend(handles=legend_elements, loc='lower left', title='Claims per Cluster', frameon=True, fontsize=8)
ax_map1.add_artist(legend)

# Add first colorbar (left-aligned for the first map)
cbar_ax1 = fig.add_axes([0.05, 0.55, 0.02, 0.3])  # [left, bottom, width, height]
sm1 = plt.cm.ScalarMappable(cmap=cmap_yellow_red, norm=norm)
cb1 = fig.colorbar(sm1, cax=cbar_ax1, orientation='vertical')
cb1.ax.tick_params(labelsize=8)
cb1.set_label('Cost ($)', fontsize=10, labelpad=10)

# Plot 2: Map of billion dollar clusters
vmin = billion_dollar['claim_sum'].min()
vmax = billion_dollar['claim_sum'].max()
norm = mcolors.LogNorm(vmin=vmin, vmax=vmax)
billion_dollar['color'] = billion_dollar['claim_sum'].apply(lambda x: cmap_yellow_red(norm(x)) if not np.isnan(x) else 'gray')

gdf_counties.plot(ax=ax_map2, color='lightgray', edgecolor='gray', alpha=0.5)
gdf_states.boundary.plot(ax=ax_map2, color='black', linewidth=0.5)

patches2 = []
for _, row in billion_dollar.iterrows():
    center = (row['jittered_longitude'], row['jittered_latitude'])
    radius = row['radius']
    color = row['color']
    circle = Circle(center, radius, edgecolor=color, facecolor=color, alpha=0.6)
    patches2.append(circle)

    ax_map2.text(
        center[0], center[1], str(row['year']),
        color='black', fontsize=9, style='italic', fontweight='bold', ha='center', va='center', zorder=5
    )

p2 = PatchCollection(patches2, match_original=True, transform=ax_map2.transData)
ax_map2.add_collection(p2)

ax_map2.set_xlim(extent[0], extent[1])
ax_map2.set_ylim(extent[2], extent[3])
ax_map2.axis('off')
ax_map2.set_title("Billion-Dollar Hyperclustered Events", fontsize=16)

# Add subplot label "b)" in the upper left
ax_map2.text(-0.1, 1, "b)", transform=ax_map2.transAxes, fontsize=14, va='top', ha='left')

# Add second colorbar (left-aligned for the second map)
cbar_ax2 = fig.add_axes([0.05, 0.1, 0.02, 0.3])  # Adjust position to align with the second map
sm2 = plt.cm.ScalarMappable(cmap=cmap_yellow_red, norm=norm)
cb2 = fig.colorbar(sm2, cax=cbar_ax2, orientation='vertical')
cb2.ax.tick_params(labelsize=8)
cb2.set_label('Cost ($)', fontsize=10, labelpad=10)

# Bar Plot 1: Historical Disasters
sns.barplot(ax=bar_axes[0], data=df2, x="Type", y="Value", palette="Greens")
bar_axes[0].set_title("Historical Disasters", fontsize=10)
bar_axes[0].set_ylabel("Claim Sum ($B)")
bar_axes[0].set_xlabel("")
bar_axes[0].set_xticklabels(bar_axes[0].get_xticklabels(), fontsize=8)

# Add subplot label "c)" in the upper left
bar_axes[0].text(-0.3, 1, "c)", transform=bar_axes[0].transAxes, fontsize=14, va='top', ha='left')

sns.barplot(ax=bar_axes[1], data=top_8_clusters, x="Event", y="claim_sum", palette="Oranges")
bar_axes[1].set_title("99.9% Hyperclusters", fontsize=10)
bar_axes[1].set_ylabel("Claim Sum ($B)")
bar_axes[1].set_xlabel("")
bar_axes[1].set_xticklabels(bar_axes[1].get_xticklabels(), fontsize=8, rotation=45)

# Add horizontal lines to the third bar plot
bar_axes[1].axhline(sum_2024/1000000000, color='red', linestyle='--', label="2024 Premiums")
bar_axes[1].axhline(sum_hist/1000000000, color='blue', linestyle='--', label="Historic Premiums")
bar_axes[1].legend(loc='upper right', fontsize=8)

# Add subplot label "d)" in the upper left
bar_axes[1].text(-0.3, 1, "d)", transform=bar_axes[1].transAxes, fontsize=14, va='top', ha='left')

if save:
    output_path = 'Plots/F2_Clusters.png'
    plt.savefig(output_path, dpi=500, bbox_inches='tight')

# Adjust layout
plt.show()

/tmp/ipykernel_2811/468505537.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cluster_centers['claim_sum'].fillna(1e4, inplace=True)  # Replace NaN with a small value (e.g., 1e-1)
/srv/conda/envs/notebook/lib/python3.12/site-packages/geopandas/plotting.py:480: UserWarning: There are no gridspecs with layoutgrids. Possibly did not call parent GridSpec with the "figure" keyword
  ax.figure.canvas.draw_idle()
/srv/conda/envs/notebook/lib/python3.12/site-packages/geopandas/plotting.py:480: UserWarning: There are no gridspecs 